In [ ]:
"""Individual Weekend Takeaway Assignment
Project Title: Password Vault – A Simple Local Password Manager
Project Description:
Build a command-line tool that lets users store, retrieve, and manage their passwords locally. The passwords should be encrypted before saving.
🛠 Features:
•	Add a new password entry (name, username, password)
•	Retrieve password using a keyword
•	Delete password entry
•	Encrypt and decrypt passwords using a basic encoding/decoding method (e.g., Fernet or base64)
•	Save/load password vault using file (JSON or text)"""



import json
import base64
from cryptography.fernet import Fernet


class PasswordEntry:
    """Represents a password entry in the vault."""

    def __init__(self, name, username, password):
        self.name = name
        self.username = username
        self.password = password  # encrypted string

    def to_dict(self):
        return {"name": self.name, "username": self.username, "password": self.password}

    @staticmethod
    def from_dict(data):
        return PasswordEntry(data["name"], data["username"], data["password"])


class PasswordVault:
    """Password vault with encryption and file persistence."""

    def __init__(self, key=None):
        self.entries = []
        # If no key provided, generate one
        self.key = key or Fernet.generate_key()
        self.cipher = Fernet(self.key)

    def encrypt(self, text: str) -> str:
        """Encrypt plain text password."""
        return self.cipher.encrypt(text.encode()).decode()

    def decrypt(self, encrypted_text: str) -> str:
        """Decrypt encrypted password."""
        return self.cipher.decrypt(encrypted_text.encode()).decode()

    def add_entry(self, name, username, password):
        encrypted_pw = self.encrypt(password)
        entry = PasswordEntry(name, username, encrypted_pw)
        self.entries.append(entry)

    def find_entry(self, keyword):
        results = []
        for e in self.entries:
            if keyword.lower() in e.name.lower() or keyword.lower() in e.username.lower():
                results.append(e)
        return results

    def delete_entry(self, name):
        before = len(self.entries)
        self.entries = [e for e in self.entries if e.name != name]
        return before != len(self.entries)

    def save_to_file(self, filepath):
        data = [e.to_dict() for e in self.entries]
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump({"key": self.key.decode(), "entries": data}, f, indent=2)

    def load_from_file(self, filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        self.key = data["key"].encode()
        self.cipher = Fernet(self.key)
        self.entries = [PasswordEntry.from_dict(e) for e in data["entries"]]


In [ ]:
def main():
    vault = PasswordVault()

    while True:
        print("\n🔐 Password Vault")
        print("1. Add new entry")
        print("2. Retrieve entry")
        print("3. Delete entry")
        print("4. Save to file")
        print("5. Load from file")
        print("0. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            name = input("Service/Website: ")
            username = input("Username: ")
            password = input("Password: ")
            vault.add_entry(name, username, password)
            print("✅ Entry added.")

        elif choice == "2":
            keyword = input("Search keyword: ")
            results = vault.find_entry(keyword)
            if not results:
                print("❌ No entry found.")
            for r in results:
                print(f"Service: {r.name}, Username: {r.username}, Password: {vault.decrypt(r.password)}")

        elif choice == "3":
            name = input("Enter service name to delete: ")
            if vault.delete_entry(name):
                print("🗑️ Entry deleted.")
            else:
                print("❌ No such entry.")

        elif choice == "4":
            filepath = input("Enter file path to save: ")
            vault.save_to_file(filepath)
            print("💾 Vault saved.")

        elif choice == "5":
            filepath = input("Enter file path to load: ")
            vault.load_from_file(filepath)
            print("📂 Vault loaded.")

        elif choice == "0":
            break

        else:
            print("Invalid choice.")


if __name__ == "__main__":
    main()



🔐 Password Vault
1. Add new entry
2. Retrieve entry
3. Delete entry
4. Save to file
5. Load from file
0. Exit
Choose an option: 1
Service/Website: gmail
Username: elsac
Password: Gimail
✅ Entry added.

🔐 Password Vault
1. Add new entry
2. Retrieve entry
3. Delete entry
4. Save to file
5. Load from file
0. Exit
Choose an option: 2
Search keyword: gmail
Service: gmail, Username: elsac, Password: Gimail

🔐 Password Vault
1. Add new entry
2. Retrieve entry
3. Delete entry
4. Save to file
5. Load from file
0. Exit
